In [ ]:
from math import factorial
import matplotlib.pyplot as plt
import numpy as np

def offeredTraffic(calls_per_hour, hours_per_call): # A0
    return calls_per_hour*hours_per_call

def calc_erlangb(n, A0):
    denom = 0
    for i in range(n+1):
        denom += (A0**i)/(factorial(i))
    E1 = ((A0**n)/factorial(n))/denom
    return E1

In [ ]:
def trafficSimulation(n_channels, numCalls, meanCallDuration):

    callDurations = np.random.exponential(scale=meanCallDuration, size=numCalls)
    callStarts = np.random.uniform(size=numCalls)
    
    callStarts.sort()
    
    # Calls = Numpy array of [[call0_start call0_end]; [call1_start call1_end];...
    calls = np.stack((callStarts, np.add(callStarts,callDurations)),axis=1)

    channelFreeTimes = np.zeros(n_channels)
    
    time = 0
    callsFailed = 0
    for i, call in enumerate(calls):
        time = call[0]
        channelFound = False
        for j in range(n_channels):
            if not(channelFound):
                if channelFreeTimes[j] < time:
                    # print("Time:", time)
                    # print("channelFreeTimes", channelFreeTimes)
                    # print("Channel {} free. Adding call time {}".format(j, call[1]))
                    channelFreeTimes[j] = call[1]
                    channelFound = True
        if channelFound == False:
            callsFailed += 1
            # print("Call {} ({}) failed.\n".format(i,call))

    return callsFailed/numCalls, callsFailed


In [ ]:
meanCallDuration = 195/3600 # 3 mins in hrs
n = 54

minNumCalls = 1
maxNumCalls = 2000
numTests = 21
simulationCallAmounts = np.concatenate((np.linspace(minNumCalls, maxNumCalls, numTests), np.linspace(2500, 6000, 8))).astype(np.int)
# simulationCallAmounts = np.linspace(100, 2000, 20).astype(np.int)
# simulationCallAmounts = np.linspace(10,10, 1).astype(np.int)

GOS_erlang = np.zeros(simulationCallAmounts.shape)
GOS_simulated = np.zeros(simulationCallAmounts.shape)

repetitionsPerSimulation = 10
print(simulationCallAmounts)

In [ ]:
for i, numCalls in enumerate(simulationCallAmounts):
    # print("{} - For {} calls in a network with {} channels:".format(i, numCalls, n))
    erlangb = calc_erlangb(n, offeredTraffic(numCalls, meanCallDuration))
    GOS_erlang[i] = erlangb*100
    # print("Erlang B = {}".format(GOS_erlang[i]))

    totalFailRate, totalFailCount = 0, 0
    for j in range(repetitionsPerSimulation):
        failRate, failsCount = trafficSimulation(n, numCalls, meanCallDuration)
        totalFailRate += failRate
        totalFailCount += failsCount
    meanFailRate = totalFailRate/repetitionsPerSimulation
    meanFailCount = totalFailCount/repetitionsPerSimulation
    GOS_simulated[i] = meanFailRate*100
    # print("Call fail rate in {} simulations = {}% (mean of {} fails in {} calls)\n".format(repetitionsPerSimulation, GOS_simulated[i], meanFailCount, numCalls))

print("Erlang B:\t\t\t Simulation:")
for i, numCalls in enumerate(simulationCallAmounts):
    print(GOS_erlang[i], "\t\t", GOS_simulated[i])

In [ ]:
# print("Erlang B:\t\t\t Simulation:")
# for i, numCalls in enumerate(simulationCallAmounts):
#     print(GOS_erlang[i], "\t\t", GOS_simulated[i])

plt.plot(simulationCallAmounts, GOS_erlang, label='Erlang B')
plt.plot(simulationCallAmounts, GOS_simulated, label='Exponential')
plt.xlabel('Number of calls')
plt.ylabel('Grade Of Service (%)')
plt.title("GOS vs number of calls made")
plt.legend()
plt.show()